In [49]:
"""
Load dadap density data, resample to correct grid, and reclassify to binary
"""


"""
this one is working, but saved rasters not showing up in S3. I am going to make a copy and test David's save function
"""

"\nthis one is working, but saved rasters not showing up in S3. I am going to make a copy and test David's save function\n"

In [50]:
import rioxarray
import rasterio
import xarray as xr
import pandas as pd
import os
import dask
import boto3
from dask.distributed import Client
import dask.array as da


In [51]:
# Print the current working directory
print("Current working directory:", os.getcwd())
print("Files in the current directory:", os.listdir())


Current working directory: /mnt/c/gis/git/AFOLU_GHG_flux_model/src/notebooks
Files in the current directory: ['.ipynb_checkpoints', 'cluster_management.ipynb', 'combine_rasters_to_10x10.ipynb', 'create_IPCC_class_and_change_rasters.ipynb', 'drainage_model.ipynb', 'LULUCF_model.ipynb', 'pp_dadap-chunks-Copy1.ipynb', 'pp_dadap-chunks-Copy2.ipynb', 'pp_dadap.ipynb', 'utilities_and_variables.ipynb']


In [52]:
tiles_test = ["00N_110E.tif"]
tiles_full = ["00N_000E.tif", "00N_010E.tif", "00N_020E.tif", "00N_030E.tif", "00N_040E.tif", "00N_040W.tif", "00N_050W.tif", "00N_060W.tif", "00N_070W.tif", "00N_080W.tif", "00N_090E.tif", "00N_090W.tif", "00N_100E.tif", "00N_110E.tif", "00N_120E.tif", "00N_130E.tif", "00N_140E.tif", "00N_150E.tif", "00N_160E.tif", "10N_000E.tif", "10N_010E.tif", "10N_010W.tif", "10N_020E.tif", "10N_020W.tif", "10N_030E.tif", "10N_040E.tif", "10N_050E.tif", "10N_050W.tif", "10N_060W.tif", "10N_070E.tif", "10N_070W.tif", "10N_080E.tif", "10N_080W.tif", "10N_090E.tif", "10N_090W.tif", "10N_100E.tif", "10N_110E.tif", "10N_120E.tif", "10N_130E.tif", "10N_140E.tif", "10S_010E.tif", "10S_020E.tif", "10S_030E.tif", "10S_040E.tif", "10S_040W.tif", "10S_050E.tif", "10S_050W.tif", "10S_060W.tif", "10S_070W.tif", "10S_080W.tif", "10S_100E.tif", "10S_110E.tif", "10S_120E.tif", "10S_130E.tif", "10S_140E.tif", "10S_150E.tif", "10S_160E.tif", "10S_170E.tif", "20N_000E.tif", "20N_010E.tif", "20N_010W.tif", "20N_020E.tif", "20N_020W.tif", "20N_030E.tif", "20N_040E.tif", "20N_050E.tif", "20N_060W.tif", "20N_070E.tif", "20N_070W.tif", "20N_080E.tif", "20N_080W.tif", "20N_090E.tif", "20N_090W.tif", "20N_100E.tif", "20N_100W.tif", "20N_110E.tif", "20N_110W.tif", "20N_120E.tif", "20N_120W.tif", "20S_010E.tif", "20S_020E.tif", "20S_030E.tif", "20S_040E.tif", "20S_050W.tif", "20S_060W.tif", "20S_070W.tif", "20S_080W.tif", "20S_110E.tif", "20S_120E.tif", "20S_130E.tif", "20S_140E.tif", "20S_150E.tif", "20S_160E.tif", "30N_010W.tif", "30N_020W.tif", "30N_030E.tif", "30N_040E.tif", "30N_050E.tif", "30N_060E.tif", "30N_070E.tif", "30N_080E.tif", "30N_080W.tif", "30N_090E.tif", "30N_090W.tif", "30N_100E.tif", "30N_100W.tif", "30N_110E.tif", "30N_110W.tif", "30N_120E.tif", "30N_120W.tif", "30S_010E.tif", "30S_020E.tif", "30S_030E.tif", "30S_060W.tif", "30S_070W.tif", "30S_080W.tif", "30S_090W.tif", "30S_110E.tif", "30S_120E.tif", "30S_130E.tif", "30S_140E.tif", "30S_150E.tif", "30S_170E.tif", "40N_000E.tif", "40N_010E.tif", "40N_010W.tif", "40N_020E.tif", "40N_030E.tif", "40N_040E.tif", "40N_050E.tif", "40N_060E.tif", "40N_070E.tif", "40N_070W.tif", "40N_080E.tif", "40N_080W.tif", "40N_090E.tif", "40N_090W.tif", "40N_100E.tif", "40N_100W.tif", "40N_110E.tif", "40N_110W.tif", "40N_120E.tif", "40N_120W.tif", "40N_130E.tif", "40N_130W.tif", "40N_140E.tif", "40S_070W.tif", "40S_080W.tif", "40S_140E.tif", "40S_160E.tif", "40S_170E.tif", "50N_000E.tif", "50N_010E.tif", "50N_010W.tif", "50N_020E.tif", "50N_030E.tif", "50N_040E.tif", "50N_050E.tif", "50N_060W.tif", "50N_070E.tif", "50N_070W.tif", "50N_080E.tif", "50N_080W.tif", "50N_090E.tif", "50N_090W.tif", "50N_100E.tif", "50N_100W.tif", "50N_110E.tif", "50N_110W.tif", "50N_120E.tif", "50N_120W.tif", "50N_130E.tif", "50N_130W.tif", "50N_140E.tif", "50N_150E.tif", "50S_060W.tif", "50S_070W.tif", "50S_080W.tif", "60N_000E.tif", "60N_010E.tif", "60N_010W.tif", "60N_020E.tif", "60N_020W.tif", "60N_030E.tif", "60N_040E.tif", "60N_050E.tif", "60N_060E.tif", "60N_060W.tif", "60N_070E.tif", "60N_070W.tif", "60N_080E.tif", "60N_080W.tif", "60N_090E.tif", "60N_090W.tif", "60N_100E.tif", "60N_100W.tif", "60N_110E.tif", "60N_110W.tif", "60N_120E.tif", "60N_120W.tif", "60N_130W.tif", "60N_140E.tif", "60N_140W.tif", "60N_150E.tif", "60N_150W.tif", "60N_160E.tif", "60N_160W.tif", "60N_170E.tif", "60N_170W.tif", "60N_180W.tif", "70N_000E.tif", "70N_010E.tif", "70N_010W.tif", "70N_020E.tif", "70N_020W.tif", "70N_030E.tif", "70N_030W.tif", "70N_040E.tif", "70N_050E.tif", "70N_060E.tif", "70N_070E.tif", "70N_080E.tif", "70N_080W.tif", "70N_090E.tif", "70N_090W.tif", "70N_100E.tif", "70N_100W.tif", "70N_110E.tif", "70N_110W.tif", "70N_120E.tif", "70N_120W.tif", "70N_130E.tif", "70N_130W.tif", "70N_140E.tif", "70N_140W.tif", "70N_150E.tif", "70N_150W.tif", "70N_160E.tif", "70N_160W.tif", "70N_170E.tif", "70N_180W.tif", "80N_020E.tif", "80N_060E.tif", "80N_070E.tif", "80N_080E.tif", "80N_090E.tif", "80N_110E.tif", "80N_120E.tif", "80N_120W.tif", "80N_130W.tif", "80N_140E.tif", "80N_140W.tif"]

# specify full or test: 
tiles = tiles_test

peatlands_uri = "s3://gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/raw/GFW_Global_Peatlands/"
dadap_uri = "s3://gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/raw/Dadap_SEA_Drainage/canal_length_data/canal_length_1km.tif"
s3_base_dir = "s3://gfw2-data/climate/AFOLU_flux_model/organic_soils/"
dadap_pattern = "dadap_density"

processed_dir = "s3:/gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/processed"
output_dir = os.path.join(processed_dir)

In [53]:
dadap_density = get_dataset(dadap_uri, "dadap_density", template=None)
test_tile = get_tile_dataset(peatlands_uri, tiles[0], "peatlands", template=None)

In [54]:
def save_and_upload_raster_10x10(data_array, out_file_name, out_folder):
    local_path = f"/tmp/{out_file_name}"
    s3_bucket = "gfw2-data"
    s3_key = f"{out_folder.strip('/')}/{out_file_name}"

    # Configure compression for the output raster
    with rasterio.Env():
        profile = data_array.profile
        profile.update(compress='lzw')

        # Save locally
        with rasterio.open(local_path, 'w', **profile) as dst:
            dst.write(data_array.read(1), 1)

    # Upload to S3
    s3_client = boto3.client("s3")
    s3_client.upload_file(local_path, s3_bucket, s3_key)
    print(f"Uploaded {out_file_name} to S3 at {s3_key}")

    # Clean up local file
    os.remove(local_path)
    print(f"Deleted local file: {local_path}")

In [ ]:
def preprocess_dadap_density():
    print("Loading dadap density raster...")
    dadap_density = get_dataset(dadap_uri, "dadap_density", template=None)

    # Load a template raster from the Hansen dataset for reference
    print("Loading template raster for CRS and resolution reference...")
    template_raster = get_tile_dataset(peatlands_uri, tiles[0], "peatlands", template=None)

    # Reproject and resample dadap_density to match template
    print("Reprojecting and resampling dadap density raster...")
    dadap_density = dadap_density.rio.reproject_match(template_raster)

    return dadap_density

dadap_density = preprocess_dadap_density()

Loading dadap density raster...
Loading template raster for CRS and resolution reference...
Reprojecting and resampling dadap density raster...


In [ ]:
def clip_and_save_dadap_density(bounds, dadap_density, is_final):
    xmin, ymin, xmax, ymax = bounds
    bounds_str = f"{xmin}_{ymin}_{xmax}_{ymax}"
    tile_id = xy_to_tile_id(bounds[0], bounds[3])  # Simplify this as needed

    # Clip the raster
    clipped = dadap_density.rio.clip_box(xmin, ymin, xmax, ymax)

    # Generate the file name
    out_file_name = f"{tile_id}_{time.strftime('%Y%m%d%H%M%S')}.tif"
    out_folder = "gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/processed"

    # Check if there is any valid data before saving
    if np.any(clipped.data != clipped.rio.nodata):
        # Save and upload using the custom function
        save_and_upload_raster_10x10(clipped, out_file_name, out_folder)
    else:
        print(f"No valid data in chunk {bounds} for tile {tile_id}. Skipping this chunk.")

In [ ]:
chunk_params = [110, -10, 120, 0, 2]  # Customize as needed
#chunk_params = [110, -6, 112, -4, 1]

# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 30:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(clip_and_save_dadap_density)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

In [ ]:
# # Convert processing to Dask delayed tasks
# tasks = [dask.delayed(process_tiles)(dadap_density, output_dir, [tile]) for tile in tiles]
# results = dask.compute(*tasks)